<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/alicia/Death_Stats_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-16 00:23:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2021-04-16 00:23:10 (10.1 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [11]:
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

url ="https://aperezbucket.s3.us-east-2.amazonaws.com/life_expectancy.csv"
spark.sparkContext.addFile(url)
life_expectancy = spark.read.csv(SparkFiles.get("life_expectancy.csv"), sep=",", header=True)

# Show DataFrame
life_expectancy.show()

+----+---------+----------+-----------------------+-----------------+
|year|     race|       sex|average_life_expectancy|age_adjusted_rate|
+----+---------+----------+-----------------------+-----------------+
|1999|All Races|Both Sexes|                   76.7|            875.6|
|1999|All Races|    Female|                   79.4|              734|
|1999|All Races|      Male|                   73.9|             1067|
|1999|    Black|Both Sexes|                   71.4|           1135.7|
|1999|    Black|    Female|                   74.7|            933.6|
|1999|    Black|      Male|                   67.8|           1432.6|
|1999|    White|Both Sexes|                   77.3|            854.6|
|1999|    White|    Female|                   79.9|            716.6|
|1999|    White|      Male|                   74.6|           1040.1|
|2000|All Races|Both Sexes|                   76.8|              869|
|2000|All Races|    Female|                   79.7|            731.4|
|2000|All Races|    

In [9]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/leading_causes_death.csv"
spark.sparkContext.addFile(url)
leading_causes_death = spark.read.csv(SparkFiles.get("leading_causes_death.csv"), sep=",", header=True)

# Show DataFrame
leading_causes_death.show()

+----+--------------------+--------------------+--------------------+------+-----------------------+
|year|               cause|          cause_name|               state|deaths|age_adjusted-death_rate|
+----+--------------------+--------------------+--------------------+------+-----------------------+
|1999|Accidents (uninte...|Unintentional inj...|             Alabama| 2,313|                   52.2|
|1999|Accidents (uninte...|Unintentional inj...|              Alaska|   294|                   55.9|
|1999|Accidents (uninte...|Unintentional inj...|             Arizona| 2,214|                   44.8|
|1999|Accidents (uninte...|Unintentional inj...|            Arkansas| 1,287|                   47.6|
|1999|Accidents (uninte...|Unintentional inj...|          California| 9,198|                   28.7|
|1999|Accidents (uninte...|Unintentional inj...|            Colorado| 1,519|                     39|
|1999|Accidents (uninte...|Unintentional inj...|         Connecticut| 1,034|               

In [27]:
leading_causes_death2 = leading_causes_death.withColumnRenamed("age_adjusted-death_rate","age_adjusted_death_rate")
leading_causes_death2.show()

+----+--------------------+--------------------+--------------------+------+-----------------------+
|year|               cause|          cause_name|               state|deaths|age_adjusted_death_rate|
+----+--------------------+--------------------+--------------------+------+-----------------------+
|1999|Accidents (uninte...|Unintentional inj...|             Alabama| 2,313|                   52.2|
|1999|Accidents (uninte...|Unintentional inj...|              Alaska|   294|                   55.9|
|1999|Accidents (uninte...|Unintentional inj...|             Arizona| 2,214|                   44.8|
|1999|Accidents (uninte...|Unintentional inj...|            Arkansas| 1,287|                   47.6|
|1999|Accidents (uninte...|Unintentional inj...|          California| 9,198|                   28.7|
|1999|Accidents (uninte...|Unintentional inj...|            Colorado| 1,519|                     39|
|1999|Accidents (uninte...|Unintentional inj...|         Connecticut| 1,034|               

In [10]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/UNdata_Export_20210414_035511888.csv"
spark.sparkContext.addFile(url)
UN_Data = spark.read.csv(SparkFiles.get("UNdata_Export_20210414_035511888.csv"), sep=",", header=True)

# Show DataFrame
UN_Data.show()

+--------------------+----+-----+----+-----+--------------------+-----------+------+---------------+
|     Country or Area|Year| Area| Sex|  Age|Cause of death (WHO)|Source Year| Value|Value Footnotes|
+--------------------+----+-----+----+-----+--------------------+-----------+------+---------------+
|United States of ...|2017|Total|Male|Total|Certain infectiou...|       2020| 35481|           null|
|United States of ...|2017|Total|Male|Total|Intestinal infect...|       2020|  3592|           null|
|United States of ...|2017|Total|Male|Total| Tuberculosis, ICD10|       2020|   321|           null|
|United States of ...|2017|Total|Male|Total|      Tetanus, ICD10|       2020|     1|           null|
|United States of ...|2017|Total|Male|Total|   Diphtheria, ICD10|       2020|     0|           null|
|United States of ...|2017|Total|Male|Total|Whooping cough, I...|       2020|     6|           null|
|United States of ...|2017|Total|Male|Total|Meningococcal inf...|       2020|    16|       

In [16]:
UN_Data2 = UN_Data.withColumnRenamed("Country or Area","Country").withColumnRenamed("Cause of death (WHO)", "Cause_of_Death_WHO")

In [17]:
UN_Data3 = UN_Data2.drop("Source Year","Value Footnotes")
UN_Data3.show()

+--------------------+----+-----+----+-----+--------------------+------+
|             Country|Year| Area| Sex|  Age|  Cause_of_Death_WHO| Value|
+--------------------+----+-----+----+-----+--------------------+------+
|United States of ...|2017|Total|Male|Total|Certain infectiou...| 35481|
|United States of ...|2017|Total|Male|Total|Intestinal infect...|  3592|
|United States of ...|2017|Total|Male|Total| Tuberculosis, ICD10|   321|
|United States of ...|2017|Total|Male|Total|      Tetanus, ICD10|     1|
|United States of ...|2017|Total|Male|Total|   Diphtheria, ICD10|     0|
|United States of ...|2017|Total|Male|Total|Whooping cough, I...|     6|
|United States of ...|2017|Total|Male|Total|Meningococcal inf...|    16|
|United States of ...|2017|Total|Male|Total|  Septicaemia, ICD10| 19603|
|United States of ...|2017|Total|Male|Total|Acute poliomyelit...|     0|
|United States of ...|2017|Total|Male|Total|      Measles, ICD10|     0|
|United States of ...|2017|Total|Male|Total|Viral h

In [28]:
from pyspark.sql.functions import monotonically_increasing_id
life_expectancy_final = life_expectancy.withColumn("id", monotonically_increasing_id() +1)
leading_causes_death_final = leading_causes_death2.withColumn("id", monotonically_increasing_id() +1)
UN_Data_final = UN_Data3.withColumn("id", monotonically_increasing_id() +1)

In [30]:
life_expectancy_final.printSchema()

root
 |-- year: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- average_life_expectancy: string (nullable = true)
 |-- age_adjusted_rate: string (nullable = true)
 |-- id: long (nullable = false)



In [29]:
leading_causes_death_final.printSchema()

root
 |-- year: string (nullable = true)
 |-- cause: string (nullable = true)
 |-- cause_name: string (nullable = true)
 |-- state: string (nullable = true)
 |-- deaths: string (nullable = true)
 |-- age_adjusted_death_rate: string (nullable = true)
 |-- id: long (nullable = false)



In [31]:
UN_Data_final.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Area: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Cause_of_Death_WHO: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- id: long (nullable = false)



In [22]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.ck7rneirj52d.us-east-2.rds.amazonaws.com:5432/Death_Database"
config = {"user":"root", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [24]:
life_expectancy_final.write.jdbc(url=jdbc_url, table='life_expectancy', mode=mode, properties=config)

In [32]:
leading_causes_death_final.write.jdbc(url=jdbc_url, table='leading_causes_death', mode=mode, properties=config)

In [26]:
UN_Data_final.write.jdbc(url=jdbc_url, table='UN_Data', mode=mode, properties=config)